### Setup

In [ ]:
!pip install pyspark

In [70]:
from collections import Counter, defaultdict
import re

from pyspark import SparkContext

In [6]:
INPUT_FILE = '/content/wiki.txt'

In [4]:
def parse_article(article: str) -> str:
    """URL статьи <tab> название статьи <tab> текст -> текст"""
    return article.split("\n")[-1]


def split_text(article_text: str) -> list[str]:
    return article_text.split(" ")

In [7]:
sc = SparkContext(master='local[*]')

### 1. Напишите программу, которая находит самое длинное слово

In [19]:
def filter_links(words: list[str]) -> list[str]:
    return [word for word in words if "http" not in word]

def longest_word(words: list[str]) -> tuple[int, str]:
    return max((len(w), w) for w in words)

In [20]:
(
    sc.textFile(INPUT_FILE)
    .map(parse_article)
    .map(split_text)
    .map(filter_links)
    .map(longest_word)
    .sortByKey(ascending=False)
    .take(1)
)

[(84,
  '10-[3-[4-(2-гидроксиэтил)пиперидин-1-ил]пропил]-N,N-диметилфенотиазин-2-сульфонамид.')]

### 2. Напишите программу, которая находит среднюю длину слов.

In [21]:
def find_text_and_words_count(words: list[str]) -> tuple[int, int]:
    return sum(len(word) for word in words), len(words)

In [25]:
rdd = (
    sc.textFile(INPUT_FILE)
    .map(parse_article)
    .map(split_text)
    .map(filter_links)
    .map(find_text_and_words_count)
    .reduce(lambda x, y: (x[0] + y[0], x[1] + y[1]))
)

rdd[0] / rdd[1]

6.514373340602216

### 3. Напишите программу, которая находит самое частоупотребляемое слово, состоящее из латинских букв.

In [38]:
def is_latin(word: str) -> bool:
    if not word:
        return False
    return all(char.isalpha() and ord(char) < 128 for char in word)


def filter_latin_words(words: list[str]) -> list[str]:
    return [word for word in words if is_latin(word=word)]


def most_common_word(words: list[str]) -> tuple[str, int]:
    if not words:
        return "", 0
    cnt = Counter(words)
    most_common = cnt.most_common(1)[0]
    return most_common[0], most_common[1]


def compare_freqs(x: tuple[str, int], y: tuple[str, int]) -> tuple[str, int]:
    if x[1] > y[1]:
      return x
    return y

In [46]:
rdd = (
    sc.textFile(INPUT_FILE)
    .map(parse_article)
    .map(split_text)
    .map(filter_links)
    .map(filter_latin_words)
    .map(most_common_word)
    .groupByKey()
    .mapValues(sum)
    .sortBy(keyfunc=lambda x: x[1], ascending=False)
    .take(1)
)

rdd[0]

('I', 1346)

### 4. Все слова, которые более чем в половине случаев начинаются с большой буквы и встречаются больше 10 раз.

In [66]:
def calculate_statistics(words: list[str]) -> list[tuple[str, tuple[int, int]]]:
    words_freq = defaultdict(lambda: (0, 0))
    for word in words:
      if not word:
        continue

      word_lower = word.lower()
      if word[0].isupper():
        words_freq[word_lower] = (words_freq[word_lower][0], words_freq[word_lower][1] + 1)
      else:
        words_freq[word_lower] = (words_freq[word_lower][0] + 1, words_freq[word_lower][1])

    if not words_freq:
      return None

    return [(k, (v[0], v[1])) for k, v in words_freq.items() if (v[1] > v[0]) and (v[0] + v[1] > 10)]

In [68]:
rdd = (
    sc.textFile(INPUT_FILE)
    .map(parse_article)
    .map(split_text)
    .map(filter_links)
    .map(calculate_statistics)
    .filter(lambda lst: bool(lst))
    .flatMap(lambda x: [(k, v) for k, v in x])
    .groupByKey()
    .mapValues(lambda values: (sum(x[0] for x in values), sum(x[1] for x in values)))
)

rdd.takeSample(num=10, withReplacement=False)

[('матье', (0, 11)),
 ('оулу', (0, 21)),
 ('стейниц', (0, 96)),
 ('футбольной', (1, 17)),
 ('примаков', (0, 12)),
 ('рыкова', (0, 11)),
 ('кольцевой', (0, 13)),
 ('ясперс', (0, 16)),
 ('мурманский', (2, 14)),
 ('клинтон', (0, 20))]

### 5. Напишите программу, которая с помощью статистики определяет устойчивые сокращения вида пр., др., ...

In [86]:
pattern = r"^[а-я]{1,3}\.$"

In [82]:
def calculate_freqs(words: list[str]) -> list[tuple[str, int]]:
    if not words:
        return []
    cnt = Counter(words)
    return list(cnt.items())

In [88]:
rdd = (
    sc.textFile(INPUT_FILE)
    .map(parse_article)
    .map(split_text)
    .map(filter_links)
    .map(calculate_freqs)
    .filter(lambda lst: bool(lst))
    .flatMap(lambda x: [(k, v) for k, v in x])
    .filter(lambda x: bool(re.match(pattern, x[0])))
    .groupByKey()
    .mapValues(sum)
    .filter(lambda x: x[1] >= 300)
)

rdd.takeSample(num=10, withReplacement=False)

[('век.', 876),
 ('год.', 1055),
 ('нет.', 311),
 ('э.', 579),
 ('км.', 350),
 ('в.', 359),
 ('чел.', 759),
 ('см.', 302),
 ('лет.', 1562),
 ('гг.', 961)]

### 6. Напишите программу, которая с помощью статистики определяет устойчивые сокращения вида т.п., н.э., ...

In [89]:
pattern = r"^[а-я]{1}\.[а-я]{1}\.$"

In [92]:
rdd = (
    sc.textFile(INPUT_FILE)
    .map(parse_article)
    .map(split_text)
    .map(filter_links)
    .map(calculate_freqs)
    .filter(lambda lst: bool(lst))
    .flatMap(lambda x: [(k, v) for k, v in x])
    .filter(lambda x: bool(re.match(pattern, x[0])))
    .groupByKey()
    .mapValues(sum)
)

rdd.takeSample(num=10, withReplacement=False)

[('д.р.', 1),
 ('д.н.', 1),
 ('т.п.', 17),
 ('в.в.', 1),
 ('и.о.', 5),
 ('п.м.', 2),
 ('р.п.', 10),
 ('т.о.', 2),
 ('а.е.', 1),
 ('с.ш.', 17)]